In [1]:
import tensorflow as tf
from keras.models import model_from_json

Using TensorFlow backend.


### Export  Classification Model

In [ ]:
# The export path contains the name and the version of the model
tf.keras.backend.set_learning_phase(0)  # Ignore dropout at inference
classify_model = tf.keras.models.load_model('/home/ubuntu/mayub/Github/learnopencv/Keras-Fine-Tuning/da_last4_layers.h5')
export_path = '../my_image_classifier/1'

# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors
# And stored with the default serving key
with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': classify_model.input},
        outputs={t.name: t for t in classify_model.outputs})

### Export Crowd Detection Models

#### If you get "Attempted to use a closed session" Restart Jupyter kernal and try again or try using `tf.keras.backend.clear_session()`

In [ ]:
## Load the MODEL Architecture File
json_file = open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/models/Model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
## Load the model for dense images and Save in H5 format
dense_model = model_from_json(loaded_model_json)
dense_model.load_weights("/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/CSRNet_MAE67.984_RMSE103.25_SFN0.838_MAPE0.173_epoch127-150.0.hdf5")
dense_model.save('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/dense_model.h5')


In [3]:
tf.keras.backend.set_learning_phase(0) 
dense_model = tf.keras.models.load_model('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/dense_model.h5')
export_path = '../crowd_detector/1'

with tf.keras.backend.get_session() as sess2:
    tf.saved_model.simple_save(sess2,export_path,inputs={'input_image': dense_model.input},outputs={t.name: t for t in dense_model.outputs})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../crowd_detector/1/saved_model.pb


#### Model 2

In [2]:
## Load the MODEL Architecture File
json_file = open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/models/Model.json', 'r')
loaded_model_json2 = json_file.read()
json_file.close()
## Load model for Sparse images
sparse_model = model_from_json(loaded_model_json2)
sparse_model.load_weights("/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/CSRNet_MAE8.31_MSE14.361_SFN0.0_MAPE0.066_epoch135-400.0.hdf5")
sparse_model.save('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/sparse_model.h5')

In [2]:
tf.keras.backend.set_learning_phase(0) 
sparse = tf.keras.models.load_model('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/weights/sparse_model.h5')
export_path = '../crowd_detector/2'

## Run this to avoid `FailedPreconditionError: Attempting to use uninitialized value` error
# tf.keras.backend.get_session().run(tf.global_variables_initializer())

with tf.keras.backend.get_session() as sess3:
    tf.saved_model.simple_save(sess3,export_path,inputs={'input_image': sparse.input},outputs={t.name: t for t in sparse.outputs})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../crowd_detector/2/saved_model.pb


In [9]:
from PIL import Image
import numpy as np
import io
import requests
import json

In [19]:
def prepare_image(img, im_type=None):
    if im_type=="classify":
        newsize = (224, 224) 
        img = img.resize(newsize) 
    #Function to load,normalize and return image 
    im = np.array(img)
    im = im/255.0
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225
    im = np.expand_dims(im,axis  = 0)
    print(str(im.shape))
    return im

In [21]:
image = Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/sparse/IMG_7.jpg')
# image = Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/sparse/IMG_15.jpg')
image = prepare_image(image)
# image = image.astype('float')

(1, 768, 1024, 3)


payload = {
    "instances": [{'input_image': image.tolist()}]
}

#sending post request to TensorFlow Serving server
r = requests.post('http://localhost:8500/v1/models/classify:predict', json=payload)
pred = json.loads(r.content.decode('utf-8'))

### Test Crowd Count Models

In [17]:
import grpc
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis import prediction_service_pb2
from tensorflow.core.framework import types_pb2
import tensorflow as tf

In [22]:
channel = grpc.insecure_channel("localhost:8500")
# channel = implementations.insecure_channel("localhost",8500)

In [23]:
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = "dense_crowd"
request.inputs["input_image"].CopyFrom(tf.make_tensor_proto(image, dtype=types_pb2.DT_FLOAT))

In [24]:
response = stub.Predict(request, timeout=60.0)
response

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connect Failed"
	debug_error_string = "{"created":"@1588950381.498980891","description":"Failed to create subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":2721,"referenced_errors":[{"created":"@1588950381.498968457","description":"Pick Cancelled","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":241,"referenced_errors":[{"created":"@1588950381.498953033","description":"Connect Failed","file":"src/core/ext/filters/client_channel/subchannel.cc","file_line":689,"grpc_status":14,"referenced_errors":[{"created":"@1588950381.498934141","description":"Failed to connect to remote host: OS Error","errno":111,"file":"src/core/lib/iomgr/tcp_client_posix.cc","file_line":205,"os_error":"Connection refused","syscall":"connect","target_address":"ipv4:127.0.0.1:8500"}]}]}]}"
>

In [13]:
predictions = tf.make_ndarray(response.outputs['y_out/Relu:0'])
print(predictions)
summ = np.sum(predictions)
print("Predicted Count: " + str(round(summ)))

[[[[8.8443421e-03]
   [8.6643193e-03]
   [6.3699763e-03]
   ...
   [1.9776708e-02]
   [2.8616395e-02]
   [2.2410918e-02]]

  [[8.2615213e-03]
   [9.0014935e-03]
   [6.4136349e-03]
   ...
   [2.5757369e-02]
   [3.3779133e-02]
   [2.5641346e-02]]

  [[8.2430672e-03]
   [8.5845552e-03]
   [7.9758959e-03]
   ...
   [3.8594414e-02]
   [3.8175978e-02]
   [2.1720767e-02]]

  ...

  [[1.4275406e-03]
   [6.7494810e-05]
   [4.2134877e-03]
   ...
   [0.0000000e+00]
   [0.0000000e+00]
   [0.0000000e+00]]

  [[7.2628185e-03]
   [6.3359253e-03]
   [2.9798727e-03]
   ...
   [0.0000000e+00]
   [0.0000000e+00]
   [0.0000000e+00]]

  [[5.0374884e-03]
   [3.2912027e-03]
   [0.0000000e+00]
   ...
   [0.0000000e+00]
   [0.0000000e+00]
   [0.0000000e+00]]]]
Predicted Count: 24.0


### Test Classification Model

In [37]:
# image = Image.open('/home/ubuntu/mayub/Github/Crowd_Detection/image_examples/sparse_crowd_01.jpg')
image = Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/dense/IMG_26.jpg')
image = prepare_image(image, im_type="classify")
# Using stub from above
request.model_spec.name = "classify"
request.inputs["input_image"].CopyFrom(tf.make_tensor_proto(image, dtype=types_pb2.DT_FLOAT))

(1, 224, 224, 3)


In [38]:
response = stub.Predict(request, timeout=60.0)
response

outputs {
  key: "dense_11/Softmax:0"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 2
      }
    }
    float_val: 0.9754768013954163
    float_val: 0.02452320232987404
  }
}
model_spec {
  name: "classify"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [39]:
predictions = tf.make_ndarray(response.outputs['dense_11/Softmax:0'])
predicted_classes = np.argmax(predictions,axis=1)
print("Predcited Class (0-dense, 1-sparse):  "+str(predicted_classes[0]))

Predcited Class (0-dense, 1-sparse):  0


In [4]:
# Take a sample picture
img_path1 = '/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/sparse/IMG_7.jpg'
img_path2 = '/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/dense/20.jpg'
# image1 = Image.open(img_path1).convert('RGB')
# image2 = Image.open(img_path2).convert('RGB')

In [25]:
# Read as binary data
data1 = open(img_path1,'rb').read()
data2 = open(img_path2,'rb').read()

In [35]:
def prepare_image(img, im_type=None):
    if im_type=="classify":
        newsize = (224, 224) 
        img = img.resize(newsize) 
    #Function to load,normalize and return image 
    im = np.array(img)
    print(im.shape)
#     print(im)
    im = im/255.0
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225
    im = np.expand_dims(im,axis  = 0)
    # print(str(im.shape()))
    return im

In [36]:
def create_tf_prediction_request():
    channel = grpc.insecure_channel("127.0.0.1:8500")
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    request = predict_pb2.PredictRequest()
    return stub,request

In [37]:
st, req = create_tf_prediction_request()
image = Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/sparse/IMG_7.jpg')
image = prepare_image(image)

req.model_spec.name = "sparse_crowd"
req.inputs["input_image"].CopyFrom(tf.make_tensor_proto(image, dtype=types_pb2.DT_FLOAT))
data = {"success": False}
response = st.Predict(req, timeout=60.0)
p_hmap = tf.make_ndarray(response.outputs['y_out/Relu:0'])
count = np.sum(p_hmap)
print(str(round(count)))

(768, 1024, 3)
54.0


In [30]:
req.model_spec.name = "dense_crowd"
response = st.Predict(req, timeout=60.0)
p_hmap = tf.make_ndarray(response.outputs['y_out/Relu:0'])
count = np.sum(p_hmap)
print(str(round(count)))

38.0


In [39]:
image2 = Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/dense/20.jpg')
image2 = image2.convert("RGB")
image2 = prepare_image(image2)

(384, 600, 3)


In [40]:
req.model_spec.name = "sparse_crowd"
req.inputs["input_image"].CopyFrom(tf.make_tensor_proto(image2, dtype=types_pb2.DT_FLOAT))
response = st.Predict(req, timeout=60.0)
p_hmap = tf.make_ndarray(response.outputs['y_out/Relu:0'])
count = np.sum(p_hmap)
print(str(round(count)))

723.0


In [41]:
req.model_spec.name = "dense_crowd"
response = st.Predict(req, timeout=60.0)
p_hmap = tf.make_ndarray(response.outputs['y_out/Relu:0'])
count = np.sum(p_hmap)
print(str(round(count)))

1579.0
